## Análisis de la metodología para la verificación y construcción de series históricas para el cálculo de la ENFICC en plantas solares fotovoltaicas.

In [1]:
import os
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Se leen todos los archivos de data